# Question-2 : Fine-tuning IndicBERT and IndicNER for telugu language using 5%(25,870 files) dataset

## Installing Required Packages

In [ ]:
!pip3 install transformers
!pip3 install datasets
!pip3 install sentencepiece
!pip3 install seqeval


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 510.5/510.5 kB 4.0 MB/s eta 0:00:00
   ---------------------------------------- 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ---------------------------------------- 365.2/365.2 kB 7.6 MB/s eta 0:00:00
   ---------------------------------------- 24.8/24.8 MB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 134.8/134.8 kB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 60.8/60.8 kB 3.2 MB/s eta 0:00:00
   ---------------------------------------- 50.4/50.4 kB 1.3 MB/s eta 0:00:00
   ---------------------------------------- 76.4/76.4 kB 2.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install accelerate -U
#NOTE: AFTER RUNNING THIS CELL,MAKESURE TO RESTART KERNEL RUNTIME AND THEN DONOT RERUN ANY CELLS WITH !pip install IN THEM,TO AVOID ERROR OCCURED WHILE EXECUTING args training arguments

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 5.7 MB/s eta 0:00:0000:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.27.2
    Uninstalling accelerate-0.27.2:
      Successfully uninstalled accelerate-0.27.2


# Question-2

# Step-1: Downloading the Namapadam dataset

In [3]:
from datasets import load_dataset
import numpy as np
lang='te'

raw_datasets = load_dataset('ai4bharat/naamapadam', lang)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset naamapadam_pr downloaded and prepared to /root/.cache/huggingface/datasets/ai4bharat___naamapadam_pr/te/1.0.0/99b5ec77eabfaa3fbff510d8cf70d7c34519486cb7dbee99ede19474ddff9b20. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

# Step-2: Analyzing the dataset

In [4]:
# print the dataset to see how it is
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 507741
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 847
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 2700
    })
})

In [5]:
named_entity_features=raw_datasets["train"].features

In [6]:
column_names = raw_datasets["train"].column_names
print(column_names)

['tokens', 'ner_tags']


In [7]:
# printing 100th row from train dataset in given Namapadam corpus
idx=100
rec=raw_datasets['train'][idx]
for w, t in zip(rec['tokens'],rec['ner_tags']):
  print('Word:{}\tNamed Entity:{}'.format(w,t))

Word:రూ	Named Entity:0
Word:.	Named Entity:0
Word:50,000	Named Entity:0
Word:డిస్కౌంట్	Named Entity:0


The named entities are labelled as integers,so let's try to get the corresponding named entities for those integers

In [8]:
labels_list = named_entity_features['ner_tags'].feature.names

map_label_to_id={} # dictionary to store the mapping between label and its corresponding integer id
for i in range(len(labels_list)):
  map_label_to_id[labels_list[i]]=named_entity_features['ner_tags'].feature.str2int(labels_list[i])
for label_id in map_label_to_id:
  print('label:{}\t Integer Id:{}'.format(label_id,map_label_to_id[label_id]))
print(map_label_to_id)
num_labels=len(labels_list)

label:O	 Integer Id:0
label:B-PER	 Integer Id:1
label:I-PER	 Integer Id:2
label:B-ORG	 Integer Id:3
label:I-ORG	 Integer Id:4
label:B-LOC	 Integer Id:5
label:I-LOC	 Integer Id:6
{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


# Step-3: Fine-tuning **IndicNER model** on the Naamapadam corpus for **TELUGU language**

In [9]:
#Loading the Pre-trained IndicNER model
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicNER")

model = AutoModelForTokenClassification.from_pretrained("ai4bharat/IndicNER")

2024-03-12 19:09:01.252405: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-12 19:09:01.252507: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-12 19:09:01.381491: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/667M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
# moving the model to Colab GPU
model=model.to("cuda")

In [10]:
# tokenize the input text and aligns the labels with the corresponding tokens,tokenizer of IndicNER will only generate tokens specific to model,but this function will also assign NER to the tokens,which allows the model to learn the relationship between the input tokens and their corresponding entity labels during training.
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], #input tokens
        padding="max_length", #Pad the tokenized inputs to a maximum length specified with the argument max_length
        truncation=True,#truncate the tokenized inputs if they exceed the maximum length
        max_length=512,#maximum length of tokenized input
        is_split_into_words=True,# We use this argument because the texts in our dataset are lists of words (with a label for each word).So,tokenizer should not further split the tokens into subwords or characters.
    )
    aligned_labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        # print("word_ids ",i," ",word_ids)
        previous_word_idx = None
        label_ids = []
        for curr_word_id in word_ids:#For each word ID, retrieve the corresponding NER tag from the labels
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically ignored in the loss function.
            if curr_word_id is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.(checks if the current token is the first token of its corresponding word,as a single word may be split into multiple tokens while tokenizing text)
            elif curr_word_id != previous_word_idx:
                label_ids.append(label[curr_word_id])
            # For the other tokens in a word(it's part of a word that has already been started by a previous token), we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.If label_all_tokens is True, then the label for the current token is assigned based on the label for that word. If it's False, then the label for non-first tokens is set to -100 to ignore them during training.
            # but by default we set the label to -100 for non-first tokens.
            else:
                label_ids.append(-100)
            previous_word_idx = curr_word_id

        aligned_labels.append(label_ids)
    tokenized_inputs["labels"] = aligned_labels
    return tokenized_inputs

In [11]:
train_dataset = raw_datasets["train"] # given train dataset
# Shuffle the train dataset
train_dataset = train_dataset.shuffle(seed=42)

# Get the first 25% of the examples
num_examples = len(train_dataset)
subset_size = int(num_examples * 0.05)
train_subset = train_dataset.select(range(subset_size))

# Tokenize and align labels for the subset
train_subset = train_subset.map(
    tokenize_and_align_labels,  # Preprocess each example using the tokenize_and_align_labels function
    batched=True,  # Use batch processing instead of individual processing
    num_proc=4,  # Number of processes to use for parallel processing
    load_from_cache_file=True,  # Load from cache file if available
    desc="Running tokenizer on 5% subset of train dataset",  # Description for the progress bar
)

Running tokenizer on 5% subset of train dataset #0:   0%|          | 0/7 [00:00<?, ?ba/s]

Running tokenizer on 5% subset of train dataset #1:   0%|          | 0/7 [00:00<?, ?ba/s]

Running tokenizer on 5% subset of train dataset #2:   0%|          | 0/7 [00:00<?, ?ba/s]

Running tokenizer on 5% subset of train dataset #3:   0%|          | 0/7 [00:00<?, ?ba/s]

In [12]:
eval_dataset = raw_datasets["validation"] #tokenizing and aligning the labels for the validation dataset
eval_dataset = eval_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on Validation dataset",
)

Running tokenizer on Validation dataset #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #2:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [13]:
test_dataset = raw_datasets["test"]
test_dataset = test_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on test dataset",
)

Running tokenizer on test dataset #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset #2:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset #3:   0%|          | 0/1 [00:00<?, ?ba/s]

### Create collator,metrics

In [14]:
#data_collator- used to take input data samples and organize them into batches suitable for efficient training of the machine learning model.
data_collator = DataCollatorForTokenClassification(tokenizer)

In [17]:
# Metrics
from datasets import load_metric
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [labels_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [labels_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value
    try:
        if results["overall_f1"]:
            final_results["f1"] = results["overall_f1"]
    except KeyError:
        pass  # If "overall_f1" is not present in results, do nothing
    #if results["overall_f1"]: final_results["f1"]=results["overall_f1"]
    return final_results

## Set Training arguments

In [18]:
args = TrainingArguments(
    output_dir='./checkpoints',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=3e-5,  # Set the learning rate
    weight_decay=0.01,    # Set the weight decay
    gradient_accumulation_steps=1,  # Number of updates steps to accumulate before performing a backward/update pass.
    logging_dir='./logs',  # Directory for storing logs
    logging_steps=10,   # Log every n updates steps
    save_strategy="epoch",  # Save model after each epoch
    evaluation_strategy="epoch",  # Evaluate after each epoch
    save_total_limit=1,  # Limit the total amount of saved models
    label_smoothing_factor=0.1,  # Apply label smoothing for regularization
)


In [19]:
trainer = Trainer(
    model=model,
    train_dataset=train_subset,#Note: CHANGE TO "train_subset",IF WORKING UNDER SUBSET OF TRAINING DATA.FOR OVERALL TRAINING DATA,USE "train_dataset"
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    args=args,
    # callbacks=[SaveCheckpointsCallback()],
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


## Train the Model

In [24]:
# Train the model
import warnings
warnings.filterwarnings("ignore")
import wandb
# Initialize wandb
wandb.init(project="your_project_name")

# Set your API key
wandb.login(key="009ac958cc59c78ab471adbb4d25b34b06a416ba")
train_result = trainer.train()
metrics = train_result.metrics

wandb: Currently logged in as: 319106410006 (jai-sri-ram). Use `wandb login --relogin` to force relogin


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Epoch,Training Loss,Validation Loss,Loc Precision,Loc Recall,Loc F1,Loc Number,Org Precision,Org Recall,Org F1,Org Number,Per Precision,Per Recall,Per F1,Per Number,Overall Precision,Overall Recall,Overall F1,Overall Accuracy,F1
1,0.623600,0.616233,0.746046,0.813218,0.778185,1044,0.702857,0.711668,0.707235,1037,0.773612,0.792914,0.783144,2004,0.748703,0.777479,0.762820,0.925824,0.762820
2,0.542000,0.621402,0.761773,0.790230,0.775740,1044,0.680987,0.718419,0.699202,1037,0.780187,0.789920,0.785024,2004,0.749643,0.771848,0.760584,0.925933,0.760584
3,0.552200,0.629297,0.764160,0.788314,0.776049,1044,0.685529,0.712633,0.698818,1037,0.771955,0.793912,0.782780,2004,0.747865,0.771848,0.759668,0.924851,0.759668


Load the checkpoint if necessary

In [ ]:
# from transformers import Trainer

# # Load the trained model and its corresponding tokenizer
# model_checkpoint = "./checkpoints"  # Path to the directory containing the checkpoints

# trainer = Trainer.from_pretrained(
#     model_checkpoint,
#     args=args,
#     train_dataset=train_dataset,
#     eval_dataset=eval_dataset,
#     tokenizer=tokenizer,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
# )


In [25]:
print(metrics)

{'train_runtime': 4493.9032, 'train_samples_per_second': 16.948, 'train_steps_per_second': 1.059, 'total_flos': 1.9901521891316736e+16, 'train_loss': 0.5740792902170473, 'epoch': 3.0}


In [26]:
trainer.log_metrics("train", train_result.metrics)
# print(metrics['overall_f1'])

***** train metrics *****
  epoch                    =        3.0
  total_flos               = 18534736GF
  train_loss               =     0.5741
  train_runtime            = 1:14:53.90
  train_samples_per_second =     16.948
  train_steps_per_second   =      1.059


## Evaluate the model using Test dataset

In [27]:
metrics = trainer.evaluate()
trainer.log_metrics("eval", metrics)
print(metrics['eval_overall_f1'])

***** eval metrics *****
  epoch                   =        3.0
  eval_LOC_f1             =      0.776
  eval_LOC_number         =       1044
  eval_LOC_precision      =     0.7642
  eval_LOC_recall         =     0.7883
  eval_ORG_f1             =     0.6988
  eval_ORG_number         =       1037
  eval_ORG_precision      =     0.6855
  eval_ORG_recall         =     0.7126
  eval_PER_f1             =     0.7828
  eval_PER_number         =       2004
  eval_PER_precision      =      0.772
  eval_PER_recall         =     0.7939
  eval_f1                 =     0.7597
  eval_loss               =     0.6293
  eval_overall_accuracy   =     0.9249
  eval_overall_f1         =     0.7597
  eval_overall_precision  =     0.7479
  eval_overall_recall     =     0.7718
  eval_runtime            = 0:01:01.07
  eval_samples_per_second =     44.208
  eval_steps_per_second   =      2.767
0.7596675099385617


In [28]:
# Evaluate the model on the test dataset
predictions, labels, metrics = trainer.predict(test_dataset)
print(metrics['test_overall_f1'])

0.7806691449814127


In [29]:
trainer.log_metrics("test", metrics)

***** test metrics *****
  test_LOC_f1             =     0.8067
  test_LOC_number         =        483
  test_LOC_precision      =     0.8143
  test_LOC_recall         =     0.7992
  test_ORG_f1             =     0.6855
  test_ORG_number         =        263
  test_ORG_precision      =     0.6791
  test_ORG_recall         =      0.692
  test_PER_f1             =      0.802
  test_PER_number         =        609
  test_PER_precision      =     0.8128
  test_PER_recall         =     0.7915
  test_f1                 =     0.7807
  test_loss               =     0.6037
  test_overall_accuracy   =     0.9335
  test_overall_f1         =     0.7807
  test_overall_precision  =     0.7865
  test_overall_recall     =     0.7749
  test_runtime            = 0:00:18.90
  test_samples_per_second =     44.813
  test_steps_per_second   =      2.804


In [30]:
num_examples = 5
for i in range(num_examples):
    print(f"Example {i+1}:")
    print("Predictions:", predictions[i])
    print("Labels:", labels[i])
    print()

Example 1:
Predictions: [[ 3.144738   -1.1663034   0.4242542  ... -0.91630787 -0.92136586
  -0.64984226]
 [ 3.4709692  -0.78675556 -0.57731515 ... -0.74683326 -0.74461263
  -0.46643263]
 [ 3.4469821  -0.5499245  -0.6034052  ... -0.94339436 -0.7769071
  -0.49291182]
 ...
 [ 0.36265033  0.13803276  3.2125306  ... -1.055425   -0.98815835
  -1.0242915 ]
 [-0.03021973  1.5678442   2.2369585  ... -1.1946025  -0.9437661
  -1.1882372 ]
 [-0.43924025  3.0054045   0.60141855 ... -1.0742683  -0.7082784
  -1.2209603 ]]
Labels: [-100    0    0 -100 -100 -100    0 -100 -100 -100 -100    1 -100 -100
    2 -100 -100 -100    2 -100    0    0 -100 -100 -100 -100 -100    0
    0    0 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -1

### Saving the model

In [31]:
trainer.save_model('CustomModel_NER')